In [1]:
import sys
sys.path.insert(0,'/Users/thomasdodd/Library/CloudStorage/OneDrive-MillfieldEnterprisesLimited/github/Omphalos')
sys.path.insert(1,'/Users/thomasdodd/Library/CloudStorage/OneDrive-MillfieldEnterprisesLimited/github')
import numpy as np
import pandas as pd
import xarray as xr

import xgboost as xgb

from importlib import reload

/Users/thomasdodd/opt/anaconda3/envs/omphalos/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
bst = xgb.Booster()
bst.load_model(fname='bfm_XGBModel_5-5_Epo-3000.json')

In [3]:
# Define all the attributes we have varied in .yaml file
attribute_names = ["Al+++","Ca++","Fe++","K+","Na+","Mg++","SiO2(aq)","Cl-"]

In [4]:
y_norm = 96.2359084
x_norms = [0.0199996777771118,0.0199998101672167,0.019999719657439,0.019999945896381,0.0199997455006281,0.0199994907051103,0.0199990220873342,0.0199997402413945]

In [5]:
# The (ab)normalization functions and scaling function
def NormalizeData(data, upper):
    return data / upper
def AbNormalizeData(data, upper):
    return data * upper
def ScaleData(data, scale_factor):
    return data * scale_factor

In [6]:
upper_x = 1.6E-2
lower_x = 0

# Brute Force Technique - Random Sampling Optimisation

In [8]:
%%time

#Finding best parameter from within the parameter space bounds set:
# samples = 100000000 # 100,000,000 runs : 5 mins for matrix, 5mins for model interrogation, 10mins work
samples = 10000000 # 10,000,000 runs : 3 mins work


x1 = np.random.uniform(lower_x, upper_x, samples)
x2 = np.random.uniform(lower_x, upper_x, samples)
x3 = np.random.uniform(lower_x, upper_x, samples)
x4 = np.random.uniform(lower_x, upper_x, samples)
x5 = np.random.uniform(lower_x, upper_x, samples)
x6 = np.random.uniform(lower_x, upper_x, samples)
x7 = np.random.uniform(lower_x, upper_x, samples)
x8 = np.random.uniform(lower_x, upper_x, samples)

for _ in x_norms:
    x1_norm = NormalizeData(x1,_)
    x2_norm = NormalizeData(x2,_)
    x3_norm = NormalizeData(x3,_)
    x4_norm = NormalizeData(x4,_)
    x5_norm = NormalizeData(x5,_)
    x6_norm = NormalizeData(x6,_)
    x7_norm = NormalizeData(x7,_)
    x8_norm = NormalizeData(x8,_)

matrix = np.ones((samples, len(attribute_names)))
matrix[:,0] = x1
matrix[:,1] = x2
matrix[:,2] = x3
matrix[:,3] = x4
matrix[:,4] = x5
matrix[:,5] = x6
matrix[:,6] = x7
matrix[:,7] = x8

matrix_norm = np.ones((samples, len(attribute_names)))
matrix_norm[:,0] = x1_norm
matrix_norm[:,1] = x2_norm
matrix_norm[:,2] = x3_norm
matrix_norm[:,3] = x4_norm
matrix_norm[:,4] = x5_norm
matrix_norm[:,5] = x6_norm
matrix_norm[:,6] = x7_norm
matrix_norm[:,7] = x8_norm

ypred_arr_norm = bst.predict(xgb.DMatrix(matrix_norm))
ypred_arr = AbNormalizeData(ypred_arr_norm, y_norm)

max_index = np.argmax(ypred_arr)
optimal_x_conditions = matrix[max_index,:]
print(f"Index point of the optimal conditions: {max_index}")
print(f"Volume Fraction of Carbonates within the Fracture: {round((ypred_arr[max_index]).astype(float),2)} %")
print("Optimal Concentrations of Aqueous Species in Carbonated Fluid:")
print(f"[Al+++] = {round((optimal_x_conditions[0]*1000),3)} mmol/kg")
print(f"[Ca++] = {round((optimal_x_conditions[1]*1000),3)} mmol/kg")
print(f"[Fe++] = {round((optimal_x_conditions[2]*1000),3)} mmol/kg")
print(f"[K+] = {round((optimal_x_conditions[3]*1000),3)} mmol/kg")
print(f"[Na+] = {round((optimal_x_conditions[4]*1000),3)} mmol/kg")
print(f"[Mg++] = {round((optimal_x_conditions[5]*1000),3)} mmol/kg")
print(f"[SiO2(aq)] = {round((optimal_x_conditions[6]*1000),3)} mmol/kg")
print(f"[Cl-] = {round((optimal_x_conditions[7]*1000),3)} mmol/kg")

Index point of the optimal conditions: 619075
Volume Fraction of Carbonates within the Fracture: 3.22 %
Optimal Concentrations of Aqueous Species in Carbonated Fluid:
[Al+++] = 0.163 mmol/kg
[Ca++] = 0.767 mmol/kg
[Fe++] = 0.605 mmol/kg
[K+] = 4.099 mmol/kg
[Na+] = 0.173 mmol/kg
[Mg++] = 11.803 mmol/kg
[SiO2(aq)] = 0.037 mmol/kg
[Cl-] = 14.787 mmol/kg
CPU times: user 3min 19s, sys: 1.25 s, total: 3min 20s
Wall time: 20.8 s
